# **신경망 모델 구성하기**

## **Import**

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## **학습을 위한 Device 얻기**

- 가능한 경우, GPU나 MPS 같은 하드웨어 가속기에서 모델을 학습하는 것이 좋음
- `torch.cuda` 또는 `torch.backends.mps`가 사용 가능하면 사용
    - 사용 불가하면 CPU 사용

In [2]:
device = ("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")

print(f"Using {device} device")

Using cuda device


## **Class 정의하기**

- 신경망 모델을 `nn.Module`을 상속 받아, `nn.Module`의 하위 클래스(subclass)로 정의
- `__init__`에서 신경망 계층 초기화
- `nn.Module`을 상속 받는 모든 클래스는 `forward` 메소드에 입력 데이터에 대한 연산을 구현!

In [3]:
class NeuralNetwork(nn.Module):
	def __init__(self):
		# 부모 클래스의 초기화 메소드 호출. 부모 클래스의 초기화 메소드를 명시적으로 호출하지 않으면 부모 클래스의 초기화 과정 수행 x
		super().__init__()
		self.flatten = nn.Flatten()
		self.linear_relu_stack = nn.Sequential(
			nn.Linear(28*28, 512),
			nn.ReLU(),
			nn.Linear(512, 512),
			nn.ReLU(),
			nn.Linear(512, 10)
		)
	
	def forward(self, x):
		x = self.flatten(x)
		logits = self.linear_relu_stack(x)

		return logits

In [4]:
model = NeuralNetwork().to(device)	# NeuralNetwork의 인스턴스를 생성 후, device로 이동

print(model)	# model 구조 출력

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


- 모델에 입력 데이터 전달
- 모델에 입력 데이터 전달하면, 백그라운드 연산과 함께 `forward` 함수 실행
    - `forward` 함수 직접 호출하면 안됨!
- 모델에 입력 전달하면 2차원 tensor 반환됨
    - dim=0에는 각 클래스에 대한 raw(원시) 예측값(10개) 들어있음
    - dim=1에는 각 출력의 개별 값 들어있음
- dim=0의 원시 예측값에 softmax 함수(출력층의 활성화 함수)를 적용하여 확률 얻음

In [14]:
X = torch.rand(1, 28, 28, device=device)	# device에 0~1 범위에서 1*28*28 크기 텐서 생성
# print(f"X: {X}")

logits = model(X)
print(f"logits: {logits}")

pred_probab = nn.Softmax(dim=1)(logits)		# 출력층의 활성화 함수 적용
print(f"pred_probab: {pred_probab}")

y_pred = pred_probab.argmax(1)		# 인자로 1을 주면, 각 행에서 최대값 index를 찾음
print(f"Predicted class: {y_pred}")

logits: tensor([[ 0.1469,  0.0500,  0.0635,  0.0863, -0.0030,  0.1396, -0.0011,  0.1272,
         -0.0198, -0.0140]], device='cuda:0', grad_fn=<AddmmBackward0>)
pred_probab: tensor([[0.1091, 0.0991, 0.1004, 0.1027, 0.0939, 0.1083, 0.0941, 0.1070, 0.0924,
         0.0929]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
Predicted class: tensor([0], device='cuda:0')


## **모델 계층(Layer)**

In [15]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


### **`nn.Flatten()`**

In [16]:
flatten = nn.Flatten()

flatten_image = flatten(input_image)
print(flatten_image.size())

torch.Size([3, 784])


### **`nn.Linear()`**

In [17]:
layer1 = nn.Linear(in_features=28*28, out_features=20)	# 784 * 20 행렬
hidden1 = layer1(flatten_image)
print(hidden1.size())

torch.Size([3, 20])


### **`nn.ReLU()`**

In [18]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.3836, -0.4894, -0.5426, -0.4817, -0.3242, -0.0568, -0.1116,  0.0920,
         -0.0570, -0.5057, -0.1809,  0.5377,  0.0836, -0.0028, -0.2601,  0.1802,
         -0.1930, -0.1139,  0.1736, -0.3348],
        [ 0.6057, -0.4301, -0.7111, -0.4037, -0.1669,  0.0030, -0.0659, -0.0966,
          0.4451, -0.5701,  0.0022,  0.3232,  0.0049,  0.2577, -0.3044,  0.2893,
         -0.3611,  0.1616,  0.3359, -0.5548],
        [ 0.2694, -0.5665, -0.2893, -0.0371,  0.1722,  0.0800, -0.4573, -0.2302,
          0.2277, -0.0686, -0.2334,  0.3932,  0.5061, -0.2767, -0.0165,  0.3034,
         -0.3946, -0.0337,  0.2778, -0.4817]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.3836, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0920, 0.0000,
         0.0000, 0.0000, 0.5377, 0.0836, 0.0000, 0.0000, 0.1802, 0.0000, 0.0000,
         0.1736, 0.0000],
        [0.6057, 0.0000, 0.0000, 0.0000, 0.0000, 0.0030, 0.0000, 0.0000, 0.4451,
         0.0000, 0.0022, 0.3232, 0.0049, 0.2577, 0.00

### **`nn.Sequential()`**

In [20]:
seq_modules = nn.Sequential(
	flatten,	# 3*784
	layer1,		# 784*20
	nn.ReLU(),
	nn.Linear(20, 10)	# 20*10
)	# 3*10

input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)
print(logits)

tensor([[-1.7599e-01, -1.8340e-01,  9.8428e-02, -1.6876e-01,  4.7279e-02,
         -7.7991e-02,  1.0766e-01,  2.2044e-01,  9.1638e-02,  4.3357e-02],
        [-2.2838e-01, -2.2089e-01,  1.0409e-01, -1.7542e-01, -3.1734e-04,
          1.0944e-02,  1.3846e-01,  2.1363e-01,  2.3289e-01,  1.2982e-01],
        [-2.0577e-01, -2.1086e-01,  1.4728e-01, -4.7553e-02, -2.0681e-02,
         -8.1054e-03,  1.5848e-01,  3.3684e-01,  1.3867e-01,  6.7316e-02]],
       grad_fn=<AddmmBackward0>)


### **`nn.Softmax()`**

In [21]:
softmax = nn.Softmax(dim=1)
print(softmax)

pred_probab = softmax(logits)
print(pred_probab)

Softmax(dim=1)
tensor([[0.0831, 0.0825, 0.1093, 0.0837, 0.1039, 0.0916, 0.1103, 0.1235, 0.1086,
         0.1035],
        [0.0769, 0.0775, 0.1073, 0.0811, 0.0966, 0.0977, 0.1110, 0.1197, 0.1220,
         0.1101],
        [0.0775, 0.0771, 0.1104, 0.0908, 0.0933, 0.0945, 0.1116, 0.1334, 0.1094,
         0.1019]], grad_fn=<SoftmaxBackward0>)


## **모델 매개변수**

In [23]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
	print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]}\n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[ 0.0101, -0.0015, -0.0089,  ..., -0.0165, -0.0307, -0.0137],
        [ 0.0263,  0.0132,  0.0239,  ...,  0.0273,  0.0277, -0.0131]],
       device='cuda:0', grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([ 0.0103, -0.0188], device='cuda:0', grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[-0.0422, -0.0102, -0.0005,  ..., -0.0416,  0.0033, -0.0229],
        [ 0.0071,  0.0206, -0.0111,  ..., -0.0316, -0.0109, -0.0365]],
       device='cuda:0', grad_fn=<SliceBa